In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken
import ollama
from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
# from graphrag.query.llm.oai.chat_openai import ChatOpenAI
# from graphrag.query.llm.oai.embedding import OpenAIEmbedding
# from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore
import ollama

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [3]:
INPUT_DIR = "/Users/luwi/Documents/Code/microsoft_graphrag_local/ragdirs/6efdb17a-fee9-4510-a83d-a7d21137888d/output"#"./inputs/operation dulce"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"/Users/luwi/Documents/Code/microsoft_graphrag_local/ragdirs/ragdir_2/output/create_base_extracted_entities.parquet")
entity_df.values

array([['<graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd">\n  <key id="d6" for="edge" attr.name="source_id" attr.type="string" />\n  <key id="d5" for="edge" attr.name="description" attr.type="string" />\n  <key id="d4" for="edge" attr.name="weight" attr.type="double" />\n  <key id="d3" for="node" attr.name="entity_type" attr.type="string" />\n  <key id="d2" for="node" attr.name="source_id" attr.type="string" />\n  <key id="d1" for="node" attr.name="description" attr.type="string" />\n  <key id="d0" for="node" attr.name="type" attr.type="string" />\n  <graph edgedefault="undirected">\n    <node id="QUESTION ANSWERING">\n      <data key="d0">EVENT</data>\n      <data key="d1">A method of using a graph to answer questions based on text</data>\n      <data key="d2">e1e4f0675d6bb822863a64b663629c0f</data>\n      <data ke

#### Read entities

In [5]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 7


,level,title,type,description,source_id,entity_type,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,BOB,PERSON,Bob is a person who likes fishing,4c79a5692000d450076d9ada58931231,PERSON,1,6,0,e2fc9fac61ed4ff8a7ca9b8d63856844,6,None,e2fc9fac61ed4ff8a7ca9b8d63856844,0,0
1,0,FISHING,EVENT,Fishing is an activity that Bob enjoys,4c79a5692000d450076d9ada58931231,EVENT,0,2,1,4da961107236468fa3fc838004a7f317,2,None,4da961107236468fa3fc838004a7f317,0,0
2,0,ANNA,,,4c79a5692000d450076d9ada58931231,None,0,2,2,ea420ed016aa4ae8ad695ddfffaf275a,2,None,ea420ed016aa4ae8ad695ddfffaf275a,0,0
3,0,ANNA'S FATHER,PERSON,"Anna's father likes fishing, which is a shared...",4c79a5692000d450076d9ada58931231,PERSON,1,1,3,fdf477753d7648b9a861df7f360c3155,1,None,fdf477753d7648b9a861df7f360c3155,0,0
4,0,FISHING ROD,EVENT,"A fishing rod is a tool used for fishing, whic...",4c79a5692000d450076d9ada58931231,None,1,1,4,bf5328b3c8fc44c9a82229287e18c1ec,1,None,bf5328b3c8fc44c9a82229287e18c1ec,0,0


#### Read relationships

In [6]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 7


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,BOB,ANNA,6.0,"Anna knows Bob, and they have some common know...",[4c79a5692000d450076d9ada58931231],76b256e1d1724a39872786618cd10817,0,6,2,8
1,BOB,FISHING,16.0,Bob likes fishing as a hobby,[4c79a5692000d450076d9ada58931231],7b6cc6c05785445097830a45a47eb064,1,6,2,8
2,BOB,ANNA'S FATHER,5.0,Bob and Anna's father share the same interest ...,[4c79a5692000d450076d9ada58931231],0b905110dbad4e20a4f1f07397683763,2,6,1,7
3,BOB,FISHING ROD,1.0,Bob uses a fishing rod to go fishing,[4c79a5692000d450076d9ada58931231],e5e261c921484dd99f3b1c2d8b2191d3,3,6,1,7
4,BOB,WATER,9.0,Bob enjoys fishing in water,[4c79a5692000d450076d9ada58931231],8e2b1700a87a4faeb65ccc2a8c436bfe,4,6,1,7


In [7]:
# NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# Please see the GRAPHRAG_CLAIM_* settings

# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

#### Read community reports

In [8]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 2


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,0,# Bob's Community\n\nThe community revolves ar...,0,2.0,Bob's Community,The community has a low impact severity rating...,The community revolves around Bob and his inte...,"[{'explanation': 'Anna knows Bob, and they hav...","{\n ""title"": ""Bob's Community"",\n ""summa...",255c11a6-a3fa-4bb8-ab2d-ebc530caad74
1,1,# Bob's Fishing Community\n\nA community cente...,0,6.5,Bob's Fishing Community,The community has a moderate impact due to the...,A community centered around Bob's fishing hobb...,[{'explanation': 'Bob's enjoyment of fishing i...,"{\n ""title"": ""Bob's Fishing Community"",\n ...",0430da86-26a0-42db-92be-5e62c39b15dd


#### Read text units

In [9]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 1


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,4c79a5692000d450076d9ada58931231,Anna knows Bob.\nBob likes fishing.\nAnna does...,19,[a01ba4ec02f4f11c88b69c625152e062],"[e2fc9fac61ed4ff8a7ca9b8d63856844, 4da96110723...","[76b256e1d1724a39872786618cd10817, 7b6cc6c0578..."


In [10]:
# !pip install langchain_ollama

In [11]:
text = '\n-Goal-\nGiven a text document that is potentially relevant to this activity and a list of entity types, identify all entities of those types from the text and all relationships among the identified entities.\n \n-Steps-\n1. Identify all entities. For each identified entity, extract the following information:\n- entity_name: Name of the entity, capitalized\n- entity_type: One of the following types: [organization,person,geo,event]\n- entity_description: Comprehensive description of the entity\'s attributes and activities\nFormat each entity as ("entity"<|><entity_name><|><entity_type><|><entity_description>)\n \n2. From the entities identified in step 1, identify all pairs of (source_entity, target_entity) that are *clearly related* to each other.\nFor each pair of related entities, extract the following information:\n- source_entity: name of the source entity, as identified in step 1\n- target_entity: name of the target entity, as identified in step 1\n- relationship_description: explanation as to why you think the source entity and the target entity are related to each other\n- relationship_strength: a numeric score indicating strength of the relationship between the source entity and target entity\n Format each relationship as ("relationship"<|><source_entity><|><target_entity><|><relationship_description><|><relationship_strength>)\n \n3. Return output in English as a single list of all the entities and relationships identified in steps 1 and 2. Use **##** as the list delimiter.\n \n4. When finished, output <|COMPLETE|>\n \n######################\n-Examples-\n######################\nExample 1:\nEntity_types: ORGANIZATION,PERSON\nText:\nThe Verdantis\'s Central Institution is scheduled to meet on Monday and Thursday, with the institution planning to release its latest policy decision on Thursday at 1:30 p.m. PDT, followed by a press conference where Central Institution Chair Martin Smith will take questions. Investors expect the Market Strategy Committee to hold its benchmark interest rate steady in a range of 3.5%-3.75%.\n######################\nOutput:\n("entity"<|>CENTRAL INSTITUTION<|>ORGANIZATION<|>The Central Institution is the Federal Reserve of Verdantis, which is setting interest rates on Monday and Thursday)\n##\n("entity"<|>MARTIN SMITH<|>PERSON<|>Martin Smith is the chair of the Central Institution)\n##\n("entity"<|>MARKET STRATEGY COMMITTEE<|>ORGANIZATION<|>The Central Institution committee makes key decisions about interest rates and the growth of Verdantis\'s money supply)\n##\n("relationship"<|>MARTIN SMITH<|>CENTRAL INSTITUTION<|>Martin Smith is the Chair of the Central Institution and will answer questions at a press conference<|>9)\n<|COMPLETE|>\n\n######################\nExample 2:\nEntity_types: ORGANIZATION\nText:\nTechGlobal\'s (TG) stock skyrocketed in its opening day on the Global Exchange Thursday. But IPO experts warn that the semiconductor corporation\'s debut on the public markets isn\'t indicative of how other newly listed companies may perform.\n\nTechGlobal, a formerly public company, was taken private by Vision Holdings in 2014. The well-established chip designer says it powers 85% of premium smartphones.\n######################\nOutput:\n("entity"<|>TECHGLOBAL<|>ORGANIZATION<|>TechGlobal is a stock now listed on the Global Exchange which powers 85% of premium smartphones)\n##\n("entity"<|>VISION HOLDINGS<|>ORGANIZATION<|>Vision Holdings is a firm that previously owned TechGlobal)\n##\n("relationship"<|>TECHGLOBAL<|>VISION HOLDINGS<|>Vision Holdings formerly owned TechGlobal from 2014 until present<|>5)\n<|COMPLETE|>\n\n######################\nExample 3:\nEntity_types: ORGANIZATION,GEO,PERSON\nText:\nFive Aurelians jailed for 8 years in Firuzabad and widely regarded as hostages are on their way home to Aurelia.\n\nThe swap orchestrated by Quintara was finalized when $8bn of Firuzi funds were transferred to financial institutions in Krohaara, the capital of Quintara.\n\nThe exchange initiated in Firuzabad\'s capital, Tiruzia, led to the four men and one woman, who are also Firuzi nationals, boarding a chartered flight to Krohaara.\n\nThey were welcomed by senior Aurelian officials and are now on their way to Aurelia\'s capital, Cashion.\n\nThe Aurelians include 39-year-old businessman Samuel Namara, who has been held in Tiruzia\'s Alhamia Prison, as well as journalist Durke Bataglani, 59, and environmentalist Meggie Tazbah, 53, who also holds Bratinas nationality.\n######################\nOutput:\n("entity"<|>FIRUZABAD<|>GEO<|>Firuzabad held Aurelians as hostages)\n##\n("entity"<|>AURELIA<|>GEO<|>Country seeking to release hostages)\n##\n("entity"<|>QUINTARA<|>GEO<|>Country that negotiated a swap of money in exchange for hostages)\n##\n##\n("entity"<|>TIRUZIA<|>GEO<|>Capital of Firuzabad where the Aurelians were being held)\n##\n("entity"<|>KROHAARA<|>GEO<|>Capital city in Quintara)\n##\n("entity"<|>CASHION<|>GEO<|>Capital city in Aurelia)\n##\n("entity"<|>SAMUEL NAMARA<|>PERSON<|>Aurelian who spent time in Tiruzia\'s Alhamia Prison)\n##\n("entity"<|>ALHAMIA PRISON<|>GEO<|>Prison in Tiruzia)\n##\n("entity"<|>DURKE BATAGLANI<|>PERSON<|>Aurelian journalist who was held hostage)\n##\n("entity"<|>MEGGIE TAZBAH<|>PERSON<|>Bratinas national and environmentalist who was held hostage)\n##\n("relationship"<|>FIRUZABAD<|>AURELIA<|>Firuzabad negotiated a hostage exchange with Aurelia<|>2)\n##\n("relationship"<|>QUINTARA<|>AURELIA<|>Quintara brokered the hostage exchange between Firuzabad and Aurelia<|>2)\n##\n("relationship"<|>QUINTARA<|>FIRUZABAD<|>Quintara brokered the hostage exchange between Firuzabad and Aurelia<|>2)\n##\n("relationship"<|>SAMUEL NAMARA<|>ALHAMIA PRISON<|>Samuel Namara was a prisoner at Alhamia prison<|>8)\n##\n("relationship"<|>SAMUEL NAMARA<|>MEGGIE TAZBAH<|>Samuel Namara and Meggie Tazbah were exchanged in the same hostage release<|>2)\n##\n("relationship"<|>SAMUEL NAMARA<|>DURKE BATAGLANI<|>Samuel Namara and Durke Bataglani were exchanged in the same hostage release<|>2)\n##\n("relationship"<|>MEGGIE TAZBAH<|>DURKE BATAGLANI<|>Meggie Tazbah and Durke Bataglani were exchanged in the same hostage release<|>2)\n##\n("relationship"<|>SAMUEL NAMARA<|>FIRUZABAD<|>Samuel Namara was a hostage in Firuzabad<|>2)\n##\n("relationship"<|>MEGGIE TAZBAH<|>FIRUZABAD<|>Meggie Tazbah was a hostage in Firuzabad<|>2)\n##\n("relationship"<|>DURKE BATAGLANI<|>FIRUZABAD<|>Durke Bataglani was a hostage in Firuzabad<|>2)\n<|COMPLETE|>\n\n######################\n-Real Data-\n######################\nEntity_types: organization,person,geo,event\nText: Page 1:\nFrom Local to Global: A Graph RAG Approach to Query-Focused Summarization: The use\nof retrieval-augmented generation (RAG) to retrieve relevant informa- tion from an external\nknowledge source enables large language models (LLMs) to answer questions over private\nand/or previously unseen document collections. However, RAG fails on global questions\ndirected at an entire text corpus, such as “What are the main themes in the dataset?”, since\nthis is inherently a query- focused summarization (QFS) task, rather than an explicit retrieval\ntask. Prior QFS methods, meanwhile, fail to scale to the quantities of text indexed by typical\nRAG systems. To combine the strengths of these contrasting methods, we propose a Graph\nRAG approach to question answering over private text corpora that scales with both the\ngenerality of user questions and the quantity of source text to be in- dexed. Our approach\nuses an LLM to build a graph-based text index in two stages: first to derive an entity\nknowledge graph from the source documents, then to pre- generate community summaries\nfor all groups of closely-related entities. Given a question, each community summary is used\nto generate a partial response, before all partial responses are again summarized in a final\nresponse to the user. For a class of global sensemaking questions over datasets in the 1\nmillion token range, we show that Graph RAG leads to substantial improvements over a na\n̈ıve RAG baseline for both the comprehensiveness and diversity of generated answers. An\nopen-source, Python-based implementation of both global and local Graph RAG approaches\nis forthcoming at https://aka.ms/graphrag.\n######################\nOutput:'

In [12]:

# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     api_key="ollama",
#     base_url="http://ollama:11434/v1",
#     model="llama3.1",
#     # callbacks=[callback_handler],
# )
# llm.invoke("Hello, world!")

# from langchain_ollama import ChatOllama
# llm = ChatOllama(
#     api_key="ollama",
#     base_url="http://localhost:11434",
#     model="llama3.1",
#     # callbacks=[callback_handler],
# )
# llm.invoke("Hello, world!")

In [13]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     api_key="ollama",
#     base_url="http://localhost:11434/v1",
#     model="llama3.1",
#     # callbacks=[callback_handler],
# )
# llm.invoke("Hello, world!")


In [14]:
# llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
# embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
import time
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key="ollama",
    base_url="http://localhost:11434/v1",
    model="llama3.1",
    # callbacks=[callback_handler],
)
# llm_1 = ChatOpenAI(
#     api_key="ollama",
#     base_url="http://ollama:11434/v1",
#     model="llama3.1",
#     # callbacks=[callback_handler],
# )

# llm_2 = ChatOpenAI(
#     api_key="ollama",
#     base_url="http://host.docker.internal:11434/v1",
#     model="llama3.1",
#     # callbacks=[callback_handler],
# )
# def run_llm(text):
#     start_time = time.time()
#     llm.invoke(text)
#     end_time = time.time()
#     print("="*10)
#     print(f"Time taken: {end_time - start_time} seconds")
#     print("="*10)
#     return llm

# run_llm("this is anohter test")


token_encoder = tiktoken.get_encoding("cl100k_base")

class OpenAICompatibleOllamaEmbedding:
    def __init__(self, model: str):
        self.model = model
        

    def __call__(self, prompt: str):
        return ollama.embeddings(model=self.model, prompt=prompt)["embedding"]

    def embed(self, prompt: str):
        return self(prompt=prompt)
    
    def embed_documents(self, texts: list[str]):
        return [self(text) for text in texts]

text_embedder = OpenAICompatibleOllamaEmbedding(model="nomic-embed-text")

# text_embedder = OpenAIEmbedding(
#     api_key=api_key,
#     api_base=None,
#     api_type=OpenaiApiType.OpenAI,
#     model=embedding_model,
#     deployment_name=embedding_model,
#     max_retries=20,
# )

### Create local search context builder

In [15]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    # covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [16]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 1,
    "top_k_relationships": 1,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [17]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [18]:
result = await search_engine.asearch("Who is Bob")
print(result.response)

search_prompt 
---Role---

You are a helpful assistant responding to questions about data in the tables provided.


---Goal---

Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format, and incorporating any relevant general knowledge.

If you don't know the answer, just say so. Do not make anything up.

Points supported by data should list their data references as follows:

"This is an example sentence supported by multiple data references [Data: <dataset name> (record ids); <dataset name> (record ids)]."

Do not list more than 5 record ids in a single reference. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:

"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Sources (15, 16), Reports (1), Entities (5, 7); Relationships (23); Claims (2, 7, 34, 46, 64, 

/Users/luwi/Documents/Code/microsoft_graphrag_local/custom_graphrag_venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [19]:
result = await search_engine.asearch("Who is Anna")
print(result.response)

search_prompt 
---Role---

You are a helpful assistant responding to questions about data in the tables provided.


---Goal---

Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format, and incorporating any relevant general knowledge.

If you don't know the answer, just say so. Do not make anything up.

Points supported by data should list their data references as follows:

"This is an example sentence supported by multiple data references [Data: <dataset name> (record ids); <dataset name> (record ids)]."

Do not list more than 5 record ids in a single reference. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:

"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Sources (15, 16), Reports (1), Entities (5, 7); Relationships (23); Claims (2, 7, 34, 46, 64, 

/Users/luwi/Documents/Code/microsoft_graphrag_local/custom_graphrag_venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [20]:
question = "Who likes Fishing"
result = await search_engine.asearch(question)
print(result.response)

search_prompt 
---Role---

You are a helpful assistant responding to questions about data in the tables provided.


---Goal---

Generate a response of the target length and format that responds to the user's question, summarizing all information in the input data tables appropriate for the response length and format, and incorporating any relevant general knowledge.

If you don't know the answer, just say so. Do not make anything up.

Points supported by data should list their data references as follows:

"This is an example sentence supported by multiple data references [Data: <dataset name> (record ids); <dataset name> (record ids)]."

Do not list more than 5 record ids in a single reference. Instead, list the top 5 most relevant record ids and add "+more" to indicate that there are more.

For example:

"Person X is the owner of Company Y and subject to many allegations of wrongdoing [Data: Sources (15, 16), Reports (1), Entities (5, 7); Relationships (23); Claims (2, 7, 34, 46, 64, 

/Users/luwi/Documents/Code/microsoft_graphrag_local/custom_graphrag_venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


In [28]:
entity_embedding_df.head()

,id,name,type,description,human_readable_id,graph_embedding,text_unit_ids,description_embedding
0,e2fc9fac61ed4ff8a7ca9b8d63856844,BOB,PERSON,Bob is a person who likes fishing,0,None,[4c79a5692000d450076d9ada58931231],"[-0.230277881026268, 1.063219666481018, -4.061..."
1,4da961107236468fa3fc838004a7f317,FISHING,EVENT,Fishing is an activity that Bob enjoys,1,None,[4c79a5692000d450076d9ada58931231],"[0.1484525203704834, 1.6802079677581787, -4.26..."
2,ea420ed016aa4ae8ad695ddfffaf275a,ANNA,,,2,None,[4c79a5692000d450076d9ada58931231],"[0.7579514980316162, 0.04374135285615921, -4.0..."
3,fdf477753d7648b9a861df7f360c3155,ANNA'S FATHER,PERSON,"Anna's father likes fishing, which is a shared...",3,None,[4c79a5692000d450076d9ada58931231],"[0.2532559037208557, 1.0358085632324219, -3.59..."
4,bf5328b3c8fc44c9a82229287e18c1ec,FISHING ROD,EVENT,"A fishing rod is a tool used for fishing, whic...",4,None,[4c79a5692000d450076d9ada58931231],"[-0.10176407545804977, 1.4610772132873535, -4...."


#### Inspecting the context data used to generate the response

In [21]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,0,BOB,Bob is a person who likes fishing,6,True
1,1,FISHING,Fishing is an activity that Bob enjoys,2,True


In [22]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,1,BOB,FISHING,Bob likes fishing as a hobby,16.0,8,1,True
1,0,BOB,ANNA,"Anna knows Bob, and they have some common know...",6.0,8,2,True
2,6,FISHING,ANNA,"Anna does not like fishing, which is different...",12.0,4,2,True


In [23]:
result.context_data["reports"].head()

,id,title,content
0,1,Bob's Fishing Community,# Bob's Fishing Community\n\nA community cente...
1,0,Bob's Community,# Bob's Community\n\nThe community revolves ar...


In [24]:
result.context_data["sources"].head()

,id,text
0,0,Anna knows Bob.\nBob likes fishing.\nAnna does...


In [25]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [26]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [27]:

question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

messages {'role': 'system', 'content': '\n---Role---\n\nYou are a helpful assistant generating a bulleted list of 5 questions about data in the tables provided.\n\n\n---Data tables---\n\n-----Conversation History-----\nturn|content\nuser|Tell me about Agent Mercer\n\n\n-----Reports-----\nid|title|content\n1|Bob\'s Fishing Community|"# Bob\'s Fishing Community\n\nA community centered around Bob\'s fishing hobby, with connections to his family and tools used in fishing.\n\n## Bob\'s fishing hobby is a central aspect of the community\n\nBob\'s enjoyment of fishing is a key factor in this community. His interest in fishing has led to connections with other entities related to fishing, such as fishing rods and licenses [Data: Entities (4, 6); Relationships (1, 3, 4)]. This suggests that the community revolves around Bob\'s hobby and its associated activities.\n\n## Anna is connected to Bob through shared knowledge\n\nAnna has some common knowledge about Bob, indicating a connection between 

/Users/luwi/Documents/Code/microsoft_graphrag_local/custom_graphrag_venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:357: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")
